Deleted tables with logs and user details. Renamed "xxx_ids" to "xxxs". Merged all contig tables to single "contigs" table.

* .tables
* .schema tablename

# UniProbe data

In [1]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///uniprobe.db')

#### Check structure of the db

In [2]:
from sqlalchemy import inspect
inspector = inspect(engine)

#### Declarative base

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float, Boolean
from sqlalchemy.orm import relationship, backref

In [5]:
import sqlalchemy.types as types

class StrIntBoolean(types.TypeDecorator):

    impl = types.Integer
    
    def process_bind_param(self, value, dialect):
        return '1' if value else '0'
    def process_result_value(self, value, dialect):
        if value is None:
            return None
        elif value == '1':
            return True
        elif value == '0':
            return False
        else:
            return None

In [6]:
class Gene(Base):
    __tablename__ = 'genes'

    gene_id = Column(Integer, primary_key=True)
    gene_name = Column(String, ForeignKey('genomic_info.gene_name'))
    species = Column(String, ForeignKey('genomic_info.species'))
    publication_id = Column(Integer, ForeignKey('publications.publication_id'))
    type = Column(String)
    extended_name = Column('gene_id_name', String)

    publication = relationship('Publication', backref=backref('genes'))
    gene_info = relationship('GeneInfo', backref=backref('genes'),
                             primaryjoin='(GeneInfo.gene_name == Gene.gene_name) &\
                                          (GeneInfo.species == Gene.species)')

    def __repr__(self):
        return '<Gene(gene_id=%s, gene_name=%s, species=%s, publication_id=%s)>' %\
            (self.gene_id, self.gene_name, self.species, self.publication_id)

In [7]:
class Contig(Base):
    __tablename__ = 'contigs'

    gene_id = Column(Integer, ForeignKey('genes.gene_id'), primary_key=True)
    kmer = Column(Integer, primary_key=True)
    enrichment_score = Column(Float)

    gene = relationship('Gene', backref=backref('contigs'))

    def __repr__(self):
        return '<Contig(gene_id=%s, kmer=%s, enrichment_score=%s)>' %\
            (self.gene_id, self.kmer, self.enrichment_score)

In [8]:
class Publication(Base):
    __tablename__ = 'publications'

    publication_id = Column(Integer, primary_key=True)
    short_ref = Column('small_ref', String)
    full_ref = Column(String)

    def __repr__(self):
        return '<Publication(publication_id=%s, short_ref=%s)>' %\
            (self.publication_id, self.short_ref)

In [9]:
class GeneInfo(Base):
    __tablename__ = 'genomic_info'

    gene_name = Column(String, primary_key=True)
    name = Column(String)
    synonyms = Column(String)
    species = Column(String, primary_key=True)
    ihop = Column(String)
    uniprot = Column(String)
    refseq_id = Column(String)
    uniq_id = Column(String)
    uniq_id_2 = Column(String)
    jaspar = Column(String)
    description = Column(String)
    domain = Column(String)

    def __repr__(self):
        return '<GeneInfo(gene_name=%s, synonyms=%s, species=%s)>' %\
            (self.gene_name, self.synonyms, self.species)

## Access data

In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [11]:
session = Session()

# From uniprobe_data.py

uniprobe_data.py takes care of defining the engine, the classes and the session. It's just just a cleaned up version of the code above.

In [1]:
from uniprobe_data import *

### Testing

In [2]:
session.query(Contig).filter(Contig.kmer == 0)[:10]

[<Contig(gene_id=0, kmer=0, enrichment_score=0.250106)>,
 <Contig(gene_id=1, kmer=0, enrichment_score=0.295985)>,
 <Contig(gene_id=2, kmer=0, enrichment_score=0.33732)>,
 <Contig(gene_id=3, kmer=0, enrichment_score=0.218833)>,
 <Contig(gene_id=4, kmer=0, enrichment_score=0.357226)>,
 <Contig(gene_id=5, kmer=0, enrichment_score=0.318749)>,
 <Contig(gene_id=6, kmer=0, enrichment_score=-0.105257)>,
 <Contig(gene_id=7, kmer=0, enrichment_score=0.234288)>,
 <Contig(gene_id=8, kmer=0, enrichment_score=0.208513)>,
 <Contig(gene_id=9, kmer=0, enrichment_score=0.266657)>]

In [3]:
session.query(Gene).filter(Gene.gene_id < 10).order_by(Gene.gene_id).all()

[<Gene(gene_id=0, gene_name=Smad3, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=1, gene_name=E2F2, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=2, gene_name=Sp4, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=3, gene_name=E2F3, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=4, gene_name=Sox14, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=5, gene_name=Tcfap2a, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=6, gene_name=Zic3, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=7, gene_name=Egr1, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=8, gene_name=Six6, species=Mus musculus, publication_id=2)>,
 <Gene(gene_id=9, gene_name=Nr2f2, species=Mus musculus, publication_id=2)>]

In [4]:
session.query(Contig).join(Gene)[:3]

[<Contig(gene_id=104, kmer=0, enrichment_score=0.16832)>,
 <Contig(gene_id=104, kmer=1, enrichment_score=0.00078)>,
 <Contig(gene_id=104, kmer=2, enrichment_score=-0.00073)>]

In [5]:
[(c, c.gene) for c in session.query(Contig).filter(Contig.kmer == 0)[:10]]

[(<Contig(gene_id=0, kmer=0, enrichment_score=0.250106)>,
  <Gene(gene_id=0, gene_name=Smad3, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=1, kmer=0, enrichment_score=0.295985)>,
  <Gene(gene_id=1, gene_name=E2F2, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=2, kmer=0, enrichment_score=0.33732)>,
  <Gene(gene_id=2, gene_name=Sp4, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=3, kmer=0, enrichment_score=0.218833)>,
  <Gene(gene_id=3, gene_name=E2F3, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=4, kmer=0, enrichment_score=0.357226)>,
  <Gene(gene_id=4, gene_name=Sox14, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=5, kmer=0, enrichment_score=0.318749)>,
  <Gene(gene_id=5, gene_name=Tcfap2a, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=6, kmer=0, enrichment_score=-0.105257)>,
  <Gene(gene_id=6, gene_name=Zic3, species=Mus musculus, publication_id=2)>),
 (<Contig(gene_id=7, kmer=0, enrichment_scor

In [6]:
[(g, g.publication, g.contigs[:10]) for g in session.query(Gene).filter(Gene.gene_id < 2)]

[(<Gene(gene_id=0, gene_name=Smad3, species=Mus musculus, publication_id=2)>,
  <Publication(publication_id=2, short_ref=Badis et al., Science 2009)>,
  [<Contig(gene_id=0, kmer=0, enrichment_score=0.250106)>,
   <Contig(gene_id=0, kmer=1, enrichment_score=0.205027)>,
   <Contig(gene_id=0, kmer=2, enrichment_score=0.125331)>,
   <Contig(gene_id=0, kmer=3, enrichment_score=0.151731)>,
   <Contig(gene_id=0, kmer=4, enrichment_score=0.10707)>,
   <Contig(gene_id=0, kmer=5, enrichment_score=0.230361)>,
   <Contig(gene_id=0, kmer=6, enrichment_score=0.114943)>,
   <Contig(gene_id=0, kmer=7, enrichment_score=0.031067)>,
   <Contig(gene_id=0, kmer=8, enrichment_score=0.150954)>,
   <Contig(gene_id=0, kmer=9, enrichment_score=0.074337)>]),
 (<Gene(gene_id=1, gene_name=E2F2, species=Mus musculus, publication_id=2)>,
  <Publication(publication_id=2, short_ref=Badis et al., Science 2009)>,
  [<Contig(gene_id=1, kmer=0, enrichment_score=0.295985)>,
   <Contig(gene_id=1, kmer=1, enrichment_score=0.

In [ ]:
[(p, p.genes) for p in session.query(Publication)]

In [6]:
for gene_info in session.query(GeneInfo):
    if len(gene_info.genes) > 1:
        print(gene_info.genes)

[<Gene(gene_id=393, gene_name=PFF0200c, species=Plasmodium falciparum, publication_id=5)>, <Gene(gene_id=478, gene_name=PFF0200c, species=Plasmodium falciparum, publication_id=15)>, <Gene(gene_id=479, gene_name=PFF0200c, species=Plasmodium falciparum, publication_id=15)>, <Gene(gene_id=500, gene_name=PFF0200c, species=Plasmodium falciparum, publication_id=15)>]
[<Gene(gene_id=63, gene_name=Hoxa3, species=Mus musculus, publication_id=2)>, <Gene(gene_id=391, gene_name=Hoxa3, species=Mus musculus, publication_id=0)>]
[<Gene(gene_id=17, gene_name=Nkx3-1, species=Mus musculus, publication_id=2)>, <Gene(gene_id=389, gene_name=Nkx3-1, species=Mus musculus, publication_id=0)>]
[<Gene(gene_id=8, gene_name=Six6, species=Mus musculus, publication_id=2)>, <Gene(gene_id=388, gene_name=Six6, species=Mus musculus, publication_id=0)>]
[<Gene(gene_id=89, gene_name=Tcf1, species=Mus musculus, publication_id=2)>, <Gene(gene_id=390, gene_name=Tcf1, species=Mus musculus, publication_id=0)>]
[<Gene(gene_id=

In [8]:
[(g, g.gene_info) for g in session.query(Gene).filter(Gene.gene_id < 10)]

[(<Gene(gene_id=0, gene_name=Smad3, species=Mus musculus, publication_id=2)>,
  <GeneInfo(gene_name=Smad3, synonyms=AU022421, Mad3, Madh3, mMad3, Mothers against decapentaplegic homolog 3, Mothers against DPP homolog 3, Smad 3, SMAD 3, species=Mus musculus)>),
 (<Gene(gene_id=1, gene_name=E2F2, species=Mus musculus, publication_id=2)>,
  <GeneInfo(gene_name=E2F2, synonyms=E2F-2, Transcription factor E2F2, species=Mus musculus)>),
 (<Gene(gene_id=2, gene_name=Sp4, species=Mus musculus, publication_id=2)>,
  <GeneInfo(gene_name=Sp4, synonyms=5730497N03Rik, HF1-b, HF-1b, Transcription factor Sp4, species=Mus musculus)>),
 (<Gene(gene_id=3, gene_name=E2F3, species=Mus musculus, publication_id=2)>,
  <GeneInfo(gene_name=E2F3, synonyms=E2F-3, E2f3a, E2F3b, mKIAA0075, Transcription factor E2F3, species=Mus musculus)>),
 (<Gene(gene_id=4, gene_name=Sox14, species=Mus musculus, publication_id=2)>,
  <GeneInfo(gene_name=Sox14, synonyms=Sox-14, Transcription factor SOX-14, species=Mus musculus)>)

In [9]:
[(g, g.type, g.gene_info) for g in session.query(Gene).filter(Gene.gene_id == 285)]

[(<Gene(gene_id=285, gene_name=Gcn4, species=Saccharomyces cerevisiae, publication_id=1)>,
  u'factor',
  None)]

In [11]:
[(g.gene_id, g.gene_info.uniprot) for g in session.query(Gene).filter(Gene.gene_id < 10)]

[(0, u'Q8BUN5'),
 (1, u'P56931'),
 (2, u'Q62445'),
 (3, u'O35261'),
 (4, u'Q04892'),
 (5, u'P34056'),
 (6, u'Q62521'),
 (7, u'P08046'),
 (8, u'Q9QZ28'),
 (9, u'P43135')]

In [12]:
[(g[0].gene_id, g[1].uniprot) for g in session.query(Gene, GeneInfo).filter(Gene.gene_id < 10)\
                                         .filter(GeneInfo.gene_name == Gene.gene_name)\
                                         .filter(GeneInfo.species == Gene.species)]

[(0, u'Q8BUN5'),
 (1, u'P56931'),
 (2, u'Q62445'),
 (3, u'O35261'),
 (4, u'Q04892'),
 (5, u'P34056'),
 (6, u'Q62521'),
 (7, u'P08046'),
 (8, u'Q9QZ28'),
 (9, u'P43135')]

In [2]:
[(c, c.gene.gene_info.gene_data.fasta) for c in session.query(Contig).filter(Contig.kmer == 0)[:10]]

[(<Contig(gene_id=0, kmer=0, enrichment_score=0.250106)>,
  u'>sp|Q8BUN5|SMAD3_MOUSE Mothers against decapentaplegic homolog 3 OS=Mus musculus GN=Smad3 PE=1 SV=2\nMSSILPFTPPIVKRLLGWKKGEQNGQEEKWCEKAVKSLVKKLKKTGQLDELEKAITTQNVNTKCITIPRSLDGRLQVSHRKGLPHVIYCRLWRWPDLHSHHELRAMELCEFAFNMKKDEVCVNPYHYQRVETPVLPPVLVPRHTEIPAEFPPLDDYSHSIPENTNFPAGIEPQSNIPETPPPGYLSEDGETSDHQMNHSMDAGSPNLSPNPMSPAHNNLDLQPVTYCEPAFWCSISYYELNQRVGETFHASQPSMTVDGFTDPSNSERFCLGLLSNVNRNAAVELTRRHIGRGVRLYYIGGEVFAECLSDSAIFVQSPNCNQRYGWHPATVCKIPPGCNLKIFNNQEFAALLAQSVNQGFEAVYQLTRMCTIRMSFVKGWGAEYRRQTVTSTPCWIELHLNGPLQWLDKVLTQMGSPSIRCSSVS'),
 (<Contig(gene_id=1, kmer=0, enrichment_score=0.295985)>,
  u'>sp|P56931|E2F2_MOUSE Transcription factor E2F2 OS=Mus musculus GN=E2f2 PE=1 SV=2\nMLRAPRTLAPATAQPTKSLPALNPTELWPSGLSSPQLCPATTATTYYTSLYTQTVPSSVALGTCLDATPHGPEGQIVRCAPAGRLPAKRKLDLEGIGRPTVPEFRTPKGKCIRVDGLPSPKTPKSPGEKTRYDTSLGLLTKKFIYLLSESEDGVLDLNWAAEVLDVQKRRIYDITNVLEGIQLIRKKSKNNIQWVGRELFEDPTRPSRQQQLGQELKELMNAEQTLDQLIQSCSLSFKHLTEDNANKKLAYVTYQDIRAVGNF

## Data exploration

In [3]:
from sqlalchemy import func
from sqlalchemy import distinct

### Repeated gene_info

In [4]:
[gene_info for gene_info in\
 session.query(GeneInfo.uniprot, func.count(GeneInfo.uniprot).label('count')).group_by(GeneInfo.uniprot)\
 if gene_info.count > 1]

[(u'', 21), (u'P08046', 2)]

In [5]:
[(gene_info, gene_info.genes) for gene_info in session.query(GeneInfo).filter(GeneInfo.uniprot == u'P08046')]

[(<GeneInfo(gene_name=Egr1, synonyms=A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, Zif268, species=Mus musculus)>,
  [<Gene(gene_id=7, gene_name=Egr1, species=Mus musculus, publication_id=2)>]),
 (<GeneInfo(gene_name=Zif268, synonyms=early growth response 1, A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, species=Mus musculus)>,
  [<Gene(gene_id=400, gene_name=Zif268, species=Mus musculus, publication_id=10)>])]

In [6]:
session.query(GeneInfo).filter(GeneInfo.gene_name == 'Zif268').all()

[<GeneInfo(gene_name=Zif268, synonyms=early growth response 1, A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, species=Mus musculus)>]

In [7]:
session.query(GeneInfo).filter(GeneInfo.gene_name == 'Egr1').all()

[<GeneInfo(gene_name=Egr1, synonyms=A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, Zif268, species=Mus musculus)>]

In [8]:
session.execute(Gene.__table__.update().where(Gene.gene_id == 400).values(gene_name='Egr1'))
session.query(GeneInfo).filter(GeneInfo.gene_name == 'Zif268').delete()
session.commit()

In [9]:
[gene_info for gene_info in\
 session.query(GeneInfo.uniprot, func.count(GeneInfo.uniprot).label('count')).group_by(GeneInfo.uniprot)\
 if gene_info.count > 1]

[(u'', 21)]

In [10]:
[(gene_info, gene_info.genes) for gene_info in session.query(GeneInfo).filter(GeneInfo.uniprot == u'P08046')]

[(<GeneInfo(gene_name=Egr1, synonyms=A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, Zif268, species=Mus musculus)>,
  [<Gene(gene_id=400, gene_name=Egr1, species=Mus musculus, publication_id=10)>,
   <Gene(gene_id=7, gene_name=Egr1, species=Mus musculus, publication_id=2)>])]

In [13]:
[gene_data.gene_info for gene_data in session.query(GeneData).filter(GeneData.uniprot == u'P08046')]

[<GeneInfo(gene_name=Egr1, synonyms=A530045N19Rik, Early growth response protein 1, egr, Egr-1, EGR-1, ETR103, Krox-1, Krox24, Krox-24, Krox-24 protein, Nerve growth factor-induced protein A, NGF1-A, NGFIA, NGFI-A, TIS8, Transcription factor Zif268, Zenk, Zfp-6, Zif268, species=Mus musculus)>]

In [14]:
[gene_info for gene_info in\
 session.query(GeneInfo.uniprot, func.count(GeneInfo.uniprot).label('count')).group_by(GeneInfo.uniprot)\
 if gene_info.count > 1]

[(u'', 21), (u'P31257', 2)]

### Types

In [28]:
print(session.query(distinct(Gene.type)).all())

[(u'factor',), (u'complex',), (u'clone',)]


In [29]:
print(session.query(func.count(Gene.gene_id)).filter(Gene.type == 'factor').all())
print(session.query(func.count(Gene.gene_id)).filter(Gene.type == 'complex').all())
print(session.query(func.count(Gene.gene_id)).filter(Gene.type == 'clone').all())

[(510,)]
[(20,)]
[(16,)]


In [31]:
print([g.gene_id for g in session.query(Gene).filter(Gene.gene_name != Gene.extended_name)])

[448, 447, 446, 445, 444, 443, 449, 450, 451, 478, 479, 486, 487, 488, 489, 490, 491, 492, 493, 494, 535, 536, 549, 550, 551]


In [32]:
print([g.type for g in session.query(Gene).filter(Gene.gene_name != Gene.extended_name)])

[u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'clone', u'complex', u'complex', u'complex', u'complex', u'complex', u'complex', u'complex', u'complex', u'complex', u'clone', u'clone', u'clone', u'clone', u'clone']


In [33]:
print([(g.gene_name, g.extended_name) for g in session.query(Gene).filter(Gene.type != 'factor')])

[(u'HLH-2_HLH-3', u'HLH-2_HLH-3'), (u'HLH-2_CND-1', u'HLH-2_CND-1'), (u'HLH-2_LIN-32', u'HLH-2_LIN-32'), (u'HLH-2_HLH-19', u'HLH-2_HLH-19'), (u'HLH-2_HLH-4', u'HLH-2_HLH-4'), (u'HLH-2_HLH-8', u'HLH-2_HLH-8'), (u'HLH-2_HLH-15', u'HLH-2_HLH-15'), (u'MXL-1_MDL-1', u'MXL-1_MDL-1'), (u'HLH-2_HLH-14', u'HLH-2_HLH-14'), (u'HLH-2_HLH-10', u'HLH-2_HLH-10'), (u'Jun_Fos', u'Jun_Fos'), (u'PFF0670w', u'PFF0670w_D1'), (u'PFD0985w', u'PFD0985w_D2'), (u'PFD0985w', u'PFD0985w_D1'), (u'PF10_0075', u'PF10_0075_D3'), (u'PF10_0075', u'PF10_0075_D2'), (u'PF10_0075', u'PF10_0075_D1'), (u'PFF0670w', u'PFF0670w_D2'), (u'PFL1900w', u'PFL1900w_D1'), (u'PFL1900w', u'PFL1900w_DLD'), (u'PFF0200c', u'PFF0200c_D1'), (u'PFF0200c', u'PFF0200c_DLD'), (u'', u'GST-CSL-NOTCH1'), (u'', u'GST-CSL'), (u'', u'GST-NOTCH1'), (u'', u'GST-NOTCH1_CSL-6His'), (u'', u'GST-NOTCH1_CSL-6His_MAML1'), (u'', u'MAML1-CSL-GST-NOTCH1'), (u'', u'MAML1-CSL-GST-NOTCH2'), (u'', u'MAML1-CSL-GST-NOTCH3'), (u'', u'MAML1-CSL-GST-NOTCH4'), (u'CLAMP', 

In [34]:
# count gene_names
print(session.query(func.count(Gene.gene_name)).filter(Gene.type == 'factor').all())
print(session.query(func.count(distinct(Gene.gene_name))).filter(Gene.type == 'factor').all())

# gene_name duplicates 
print([g for g in session.query(Gene.gene_name, func.count(Gene.gene_name).label('count'))\
                       .group_by(Gene.gene_name).filter(Gene.type == 'factor') if g.count > 1])

[(510,)]
[(494,)]
[(u'Cbf1', 2), (u'Ecm22', 2), (u'Ehf', 2), (u'Elf3', 2), (u'Gabpa', 2), (u'HLH-1', 2), (u'Hoxa3', 2), (u'Nkx3-1', 2), (u'PFF0200c', 2), (u'Rap1', 2), (u'Sfpi1', 2), (u'Six4', 2), (u'Six6', 2), (u'Sox4', 2), (u'Spdef', 2), (u'Tcf1', 2)]


In [35]:
repeated_gene_names = [g.gene_name for g in session.query(Gene.gene_name, func.count(Gene.gene_name).label('count'))\
                       .group_by(Gene.gene_name).filter(Gene.type == 'factor') if g.count > 1]

print([(g.gene_name, g.species, g.publication_id) for g in session.query(Gene).order_by(Gene.gene_name)\
      .filter(Gene.type == 'factor').filter(Gene.gene_name.in_(repeated_gene_names)).all()])

[(u'Cbf1', u'Saccharomyces cerevisiae', 8), (u'Cbf1', u'Saccharomyces cerevisiae', 1), (u'Ecm22', u'Saccharomyces cerevisiae', 1), (u'Ecm22', u'Saccharomyces cerevisiae', 16), (u'Ehf', u'Mus musculus', 2), (u'Ehf', u'Mus musculus', 13), (u'Elf3', u'Mus musculus', 2), (u'Elf3', u'Mus musculus', 13), (u'Gabpa', u'Mus musculus', 2), (u'Gabpa', u'Mus musculus', 13), (u'HLH-1', u'Caenorhabditis elegans', 3), (u'HLH-1', u'Caenorhabditis elegans', 25), (u'Hoxa3', u'Mus musculus', 2), (u'Hoxa3', u'Mus musculus', 0), (u'Nkx3-1', u'Mus musculus', 2), (u'Nkx3-1', u'Mus musculus', 0), (u'PFF0200c', u'Plasmodium falciparum', 5), (u'PFF0200c', u'Plasmodium falciparum', 15), (u'Rap1', u'Saccharomyces cerevisiae', 8), (u'Rap1', u'Saccharomyces cerevisiae', 1), (u'Sfpi1', u'Mus musculus', 2), (u'Sfpi1', u'Mus musculus', 13), (u'Six4', u'Mus musculus', 0), (u'Six4', u'Drosophila melanogaster', 19), (u'Six6', u'Mus musculus', 2), (u'Six6', u'Mus musculus', 0), (u'Sox4', u'Mus musculus', 2), (u'Sox4', u'H

In [36]:
list(session.query(GeneInfo))[:10]

[<GeneInfo(gene_name=PFF0200c, synonyms=NONE, species=Plasmodium falciparum)>,
 <GeneInfo(gene_name=PF14_0633, synonyms=NONE, species=Plasmodium falciparum)>,
 <GeneInfo(gene_name=Cgd2_3490, synonyms=NONE, species=Cryptosporidium parvum)>,
 <GeneInfo(gene_name=CEH-22, synonyms="F29F11.5, Homeobox protein ceh-22", species=Caenorhabditis elegans)>,
 <GeneInfo(gene_name=Oct-1, synonyms="POU2F1, NF-A1, OCT1, Oct-1, Octamer-binding transcription factor 1, OTF1, OTF-1", species=Homo sapiens)>,
 <GeneInfo(gene_name=Alx3, synonyms=Homeobox protein aristaless-like 3, Proline-rich transcription factor ALX3, species=Mus musculus)>,
 <GeneInfo(gene_name=Alx4, synonyms=ALX-4, Aristaless-like 4, Homeobox protein aristaless-like 4, species=Mus musculus)>,
 <GeneInfo(gene_name=Arx, synonyms=Aristaless-related homeobox, Homeobox protein ARX, species=Mus musculus)>,
 <GeneInfo(gene_name=Bapx1, synonyms=Bagpipe homeobox protein homolog 1, Homeobox protein Nkx-3.2, NKX3.2, Nkx-3.2, Nkx3-2, Nkx3b, species=

In [37]:
print(session.query(func.count(distinct(GeneInfo.gene_name))).all())

# duplicates
print([g for g in session.query(GeneInfo.gene_name, func.count(GeneInfo.gene_name).label('count'))\
                       .group_by(GeneInfo.gene_name) if g.count > 1])

[(493,)]
[(u'Six4', 2), (u'Sox4', 2)]


In [38]:
print(session.query(GeneInfo).filter(GeneInfo.gene_name.in_(['Six4', 'Sox4'])).all())

[<GeneInfo(gene_name=Six4, synonyms=CG3871, Dmel\CG3871, Dsix4, DSix4, D-six4, D-Six4, six4, six4/5, species=Drosophila melanogaster)>, <GeneInfo(gene_name=Six4, synonyms=AI047561, Arec3, AREC3, Homeobox protein SIX4, Sine oculis homeobox homolog 4, Skeletal muscle-specific ARE binding protein AREC3, TrexBF, species=Mus musculus)>, <GeneInfo(gene_name=Sox4, synonyms=Transcription factor SOX-4, OTTHUMP00000039358, SRY-related HMG-box gene 4, ecotropic viral integration site 16, EVI16
, species=Homo sapiens)>, <GeneInfo(gene_name=Sox4, synonyms=AA682046, Sox-4, Transcription factor SOX-4, species=Mus musculus)>]


In [39]:
g_Gene = set(session.query(distinct(Gene.gene_name)).all())
g_GeneInfo = set(session.query(distinct(GeneInfo.gene_name)).all())

print(g_Gene - g_GeneInfo)
print(g_GeneInfo - g_Gene)

set([(u'HLH-2_HLH-14',), (u'',), (u'HLH-2_HLH-15',), (u'HLH-2_CND-1',), (u'HLH-2_HLH-19',), (u'MXL-1_MDL-1',), (u'Jun_Fos',), (u'HLH-2_HLH-10',), (u'HLH-2_HLH-3',), (u'HLH-2_HLH-4',), (u'Gcn4',), (u'HLH-2_LIN-32',), (u'HLH-2_HLH-8',)])
set([])


In [40]:
print(session.query(Gene).filter(Gene.gene_name == 'Gcn4').all())

[<Gene(gene_id=285, gene_name=Gcn4, species=Saccharomyces cerevisiae, publication_id=1)>]


In [41]:
print(session.query(GeneInfo).filter(GeneInfo.gene_name == 'Gcn4').all())

[]


In [5]:
for g in session.query(distinct(Gene.species)).filter(Gene.type == 'factor'):
    print(g[0])

Plasmodium falciparum
Cryptosporidium parvum
Caenorhabditis elegans
Saccharomyces cerevisiae
Homo sapiens
Mus musculus
Vibrio harveyi
Patiria miniata
Strongylocentrotus purpuratus
Drosophila melanogaster
Acanthamoeba castellanii
Allomyces macrogynus
Ashbya gossypii
Aspergillus nidulans
Kluyveromyces lactis
Monosiga brevisollis
Mycosphaerella graminicola
Nematostella vectensis
Trichoplax adhaerens
Tuber melanosporum
Arabidopsis thaliana


In [43]:
for s in session.query(distinct(Gene.species)).filter(Gene.type == 'factor'):
    print("%s\t%s" % (session.query(func.count(Gene.gene_id)).filter(Gene.species == s[0]).all()[0][0], s[0]))

32	Plasmodium falciparum
1	Cryptosporidium parvum
37	Caenorhabditis elegans
119	Saccharomyces cerevisiae
24	Homo sapiens
300	Mus musculus
1	Vibrio harveyi
1	Patiria miniata
2	Strongylocentrotus purpuratus
16	Drosophila melanogaster
1	Acanthamoeba castellanii
2	Allomyces macrogynus
1	Ashbya gossypii
2	Aspergillus nidulans
1	Kluyveromyces lactis
1	Monosiga brevisollis
1	Mycosphaerella graminicola
1	Nematostella vectensis
1	Trichoplax adhaerens
1	Tuber melanosporum
1	Arabidopsis thaliana


In [44]:
for s in session.query(distinct(Gene.species)).filter(Gene.type == 'factor'):
    if session.query(func.count(Gene.gene_id)).filter(Gene.species == s[0]).all()[0][0] < 3:
        print("%s\t\t\t%s" % (session.query(Gene.gene_name).filter(Gene.species == s[0]).all(), s[0]))

[(u'Cgd2_3490',)]			Cryptosporidium parvum
[(u'LuxR',)]			Vibrio harveyi
[(u'PmTbr',)]			Patiria miniata
[(u'SpTbr',), (u'FoxM_Spur',)]			Strongylocentrotus purpuratus
[(u'ACA1_126960',)]			Acanthamoeba castellanii
[(u'AMAG_02766',), (u'AMAG_00796',)]			Allomyces macrogynus
[(u'AEL327Wp',)]			Ashbya gossypii
[(u'ANID_04985',), (u'ANID_04521',)]			Aspergillus nidulans
[(u'KLLA0F08206p',)]			Kluyveromyces lactis
[(u'FoxN1-4_Mbre',)]			Monosiga brevisollis
[(u'Fox3_Mgra',)]			Mycosphaerella graminicola
[(u'FoxN2-3_Nvec',)]			Nematostella vectensis
[(u'FoxN2-3_Tadh',)]			Trichoplax adhaerens
[(u'Fox3_Tmel',)]			Tuber melanosporum
[(u'LUX',)]			Arabidopsis thaliana


In [45]:
session.query(Gene.publication_id).filter(Gene.gene_name == 'LUX').all()

[(26)]

In [46]:
session.query(Gene).filter(Gene.publication_id == 26).all()

[<Gene(gene_id=552, gene_name=LUX, species=Arabidopsis thaliana, publication_id=26)>]

In [47]:
session.query(GeneInfo.synonyms).filter(GeneInfo.species == 'Arabidopsis thaliana').all()

[(u'LUX ARRHYTHMO; PCL1; PHYTOCLOCK 1')]

In [48]:
for s in session.query(distinct(Gene.species)).filter(Gene.type == 'factor'):
    if session.query(func.count(Gene.gene_id)).filter(Gene.species == s[0]).all()[0][0] < 3:
        print("%s\t\t\t%s\t\t\t%s" % (session.query(Gene.gene_name).filter(Gene.species == s[0]).all(),\
                              session.query(GeneInfo.synonyms).filter(GeneInfo.species == s[0]).all(),\
                              s[0]))

[(u'Cgd2_3490',)]			[(u'NONE',)]			Cryptosporidium parvum
[(u'LuxR',)]			[(None,)]			Vibrio harveyi
[(u'PmTbr',)]			[(u'',)]			Patiria miniata
[(u'SpTbr',), (u'FoxM_Spur',)]			[(u'FoxM',), (u'',)]			Strongylocentrotus purpuratus
[(u'ACA1_126960',)]			[(u'A.cas Fox',)]			Acanthamoeba castellanii
[(u'AMAG_02766',), (u'AMAG_00796',)]			[(u'',), (u'',)]			Allomyces macrogynus
[(u'AEL327Wp',)]			[(u'A.gos Fox3',)]			Ashbya gossypii
[(u'ANID_04985',), (u'ANID_04521',)]			[(u'fhpA, A.nid Fox4',), (u'',)]			Aspergillus nidulans
[(u'KLLA0F08206p',)]			[(u'K.lac Fox3',)]			Kluyveromyces lactis
[(u'FoxN1-4_Mbre',)]			[(u'M.bre FoxN1/4',)]			Monosiga brevisollis
[(u'Fox3_Mgra',)]			[(u'M.gra Fox3',)]			Mycosphaerella graminicola
[(u'FoxN2-3_Nvec',)]			[(u'N.vec FoxN2/3',)]			Nematostella vectensis
[(u'FoxN2-3_Tadh',)]			[(u'T.adh FoxN2/3',)]			Trichoplax adhaerens
[(u'Fox3_Tmel',)]			[(u'T.mel Fox3',)]			Tuber melanosporum
[(u'LUX',)]			[(u'LUX ARRHYTHMO; PCL1; PHYTOCLOCK 1',)]			Arabidopsis thali

In [49]:
[(g, g.type, g.contigs) for g in session.query(Gene).filter(Gene.gene_id == 388)]

[(<Gene(gene_id=388, gene_name=Six6, species=Mus musculus, publication_id=0)>,
  u'factor',
  [])]